<a href="https://colab.research.google.com/github/Alokik-29/cow_det/blob/main/cow_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lukex9442/indian-bovine-breeds")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/indian-bovine-breeds


In [101]:


# Download latest version
path2 = kagglehub.dataset_download("miadul/animal-image-classification-5-species")

print("Path to dataset files:", path2)

Path to dataset files: /kaggle/input/animal-image-classification-5-species


In [102]:
path
path2

'/kaggle/input/animal-image-classification-5-species'

In [103]:
import os,shutil
from sklearn.model_selection import train_test_split
import random

cow_path=path
animal_path=path2
base_dir =base_dir = "/content/final_dataset"
classes = ["Cow", "NotCow"]

In [104]:
for split in ["train","test"]:
  for cls in classes:
    os.makedirs(os.path.join(base_dir,split,cls),exist_ok=True)


In [105]:
# Collect cow images (only from Indian Bovine dataset)
cow_images=[]
for root, dirs, files in os.walk(cow_path):
  for f in files:
    if f.lower().endswith(("jpg","jpeg","png")):
            cow_images.append(os.path.join(root, f))

random.shuffle(cow_images)
cow_images = cow_images[:629]

In [106]:
# Collect non-cow images (exclude "Cow" class from the 5 species dataset)
not_cow_images = []
for root, dirs, files in os.walk(animal_path):
    if "Cow" in root:   # Skip cow class in 5 species dataset
        continue
    for f in files:
        if f.lower().endswith(("jpg","jpeg","png")):
            not_cow_images.append(os.path.join(root, f))

In [107]:
print("Total Cow images:", len(cow_images))
print("Total Not-Cow images:", len(not_cow_images))

Total Cow images: 629
Total Not-Cow images: 629


In [108]:
# Train/Test Split
def split_and_copy(images, cls_name):
    train, test = train_test_split(images, test_size=0.2, random_state=42)

    for img in train:
        shutil.copy(img, os.path.join(base_dir, "train", cls_name))
    for img in test:
        shutil.copy(img, os.path.join(base_dir, "test", cls_name))

split_and_copy(cow_images, "Cow")
split_and_copy(not_cow_images, "NotCow")

print("Dataset prepared at:", base_dir)

Dataset prepared at: /content/final_dataset


In [109]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [110]:
# Image size for transfer learning (224x224 works well with ResNet/MobileNet)
img_size = 224
batch_size = 32

train_transforms=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder("/content/final_dataset/train", transform=train_transforms)
test_data = datasets.ImageFolder("/content/final_dataset/test", transform=test_transforms)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

class_names = train_data.classes
print("Classes:", class_names)

Classes: ['Cow', 'NotCow']


In [111]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained model
model = models.resnet18(pretrained=True)

# Replace final layer for binary classification first
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)

# Freeze backbone
for name, param in model.named_parameters():
    if "fc" not in name:
        param.requires_grad = False
    else:
        param.requires_grad = True  # ensure final layer is trainable

# Update optimizer to train only the final layer
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

# Move model to device
model = model.to(device)

In [112]:
criterion = nn.BCEWithLogitsLoss()  # since we use single output neuron
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [113]:
def train_model(model,train_loader,test_loader,epochs=5):
  for epoch in range(epochs):
    model.train()
    running_loss,correct,total=0.0,0,0

    for images,labels in train_loader:
      images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

      optimizer.zero_grad()
      outputs=model(images)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss +=loss.item()
      preds =torch.sigmoid(outputs)>0.5
      correct+= (preds==labels.byte()).sum().item()
      total +=  labels.size(0)

    train_acc=100* correct/total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    #Validation
    model.eval()
    correct,total=0,0
    with torch.no_grad():
      for images,labels in test_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
        outputs=model(images)
        preds = torch.sigmoid(outputs) > 0.5
        correct += (preds == labels.byte()).sum().item()
        total += labels.size(0)
    val_acc= 100*correct/total
    print(f"Validation Acc: {val_acc:.2f}%\n")

In [ ]:
train_model(model,train_loader,test_loader,epochs=5)